### Part8 - Deep Learning - Convolutional Neural Networks(CNN)

### Lecture note

- reference : http://taewan.kim/post/cnn/

#### image

- image를 숫자로 표현하고 색을 구하고 표현


#### convolution

- convolution is a combined integration of the two functions and it shows you how one function modifies the other or modifies the shape
- 자세한건 이미지 참고, 논문 참고
- filter?
  - feature detector를 이용하여 input image를 순회하여 feature map을 완성
  - 완성된 map에서 가장 높은 값을 지니는 부분을 확인한다.
  
- we create many feature maps to obtain our first convolution layer
- filter를 이용하면 다양한 이미지 결과를 이끌어 낼 수 있다.(이미지 참고)

#### ReLU layer

- Rectifier
  - image are highly non-linear